# Importing Libraries

In [1]:
# Importing required librariesz
from requests import get
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep, time
from random import randint
from IPython.display import clear_output
import pandas as pd

# Building Scraper

In [ ]:
# Creating lists for storing Data
Symbols = []
Names = []
Categories =[]
Prices = []
DayPercents = []
WeekPrecets = []
MarketCap = []
VDollars = []
VCrypto =[]
CSupply = []
WeekGraph = []

# Preparing the monitoring of loop
start_time = time()
requests = 0

# Base URL
base_url = 'https://coinmarketcap.com/?page='

# Iterating through pages
pages = [str(i) for i in range(1,54)]
for page in pages:
    browser=webdriver.Chrome()
    browser.get(base_url + page)
    
    # Smooth scorlling of pgae (lazy loading)
    total_height = int(browser.execute_script("return document.body.scrollHeight"))
    for i in range(1, total_height, 500):
        browser.execute_script("window.scrollTo(0, {});".format(i))
        sleep(4) # Waiting will help load all data    
    
    # Pause the loop
    sleep(randint(8,10))
    
    # Monitot the request
    requests += 1
    elapsed_time = time() - start_time
    clear_output(wait = True)
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
     
    # Break the loop if the number of requests is greater than expected
    if requests > 52:
        warn('Number of requests was greater than expected.')
        break       
    
    # Parse the content of the request with BeautifulSoup
    html = browser.page_source
    html_soup = BeautifulSoup(html,'html.parser')

    # Select all the cryptcurrency data
    crypto_collection = html_soup.find('div', class_ = 'tableWrapper___3utdq cmc-table-homepage-wrapper___22rL4')
    crypto_data = crypto_collection.tbody.find_all('tr')

    dp = 1
    # Extract data for individula row 
    for data in crypto_data:
        
        # Extract Symbol
        sym = data.find_all('td')[2].find('img').attrs['src']
        Symbols.append(sym)
         
        # Extract Name
        name = data.find('div',class_ = 'sc-16r8icm-0 sc-1teo54s-1 cPNAgw').p.text
        Names.append(name)
       
        # Extract Category
        category = data.find('div', class_ = 'sc-1teo54s-2 fZIJcI').p.text
        Categories.append(category)
      
        # Extracting Price
        price = float(data.find_all('td')[3].text.replace("<","").lstrip('$').replace(",",""))
        Prices.append(price)        
        
        # Extracting # 24H% variation
        dayVar = data.find_all('td')[4]
        if dayVar.find('span', class_ = 'icon-Caret-up') is not None:
            dayVar = data.find_all('td')[4].find_all('span')[0].text + str(" %Up")
        elif dayVar.find('span', class_ = 'icon-Caret-down') is not None:
            dayVar =  data.find_all('td')[4].find_all('span')[0].text + str(" %Down")
        else:
            dayVar = 0 
        DayPercents.append(dayVar)
                
        # Extracting 7d%
        week = data.find_all('td')[5] 
        if week.find('span', class_ = 'icon-Caret-up') is not None:
            week = data.find_all('td')[5].find_all('span')[0].text  + str(" %Up")
        elif week.find('span', class_ = 'icon-Caret-down') is not None:
            week =  data.find_all('td')[5].find_all('span')[0].text + str(" %Down")
        else:
            week = 0
        WeekPrecets.append(week)
    
        # Extracting Market Cap
        if data.find_all('td')[6].p is not None:
            mcap = float(data.find_all('td')[6].p.find('span', class_ = 'ggzdxp-1 ftvydZ' ).text.lstrip('$').replace(",",""))
            MarketCap.append(mcap)
        else:
            MarketCap.append('nan')
  

        # Extracting Volume 24h in $
        if data.find_all('td')[7].p is not None:
            vDollars = float(data.find_all('td')[7].find_all('p')[0].text.lstrip('$').replace(",","")) 
            VDollars.append(vDollars)
        else:
            VDollars.append(0)
            
        # Extracitng  Volume 24h in crypto
        if data.find_all('td')[7].p is not None:
            vCrypto = float(data.find_all('td')[7].find_all('p')[1].text.lstrip('$').replace(category.upper(),"").replace(category.lower(),"").replace(",","")) 
            VCrypto.append(vCrypto)
        else:
             VCrypto.append(0)
        
        # Extracting circulaitng Supply
        if data.find_all('td')[8].p is not None:
            csupply = float(data.find_all('td')[8].p.text.rstrip(category).replace(",",""))
            CSupply.append(csupply)
        else:
            CSupply.append(0)
        
        # Extracing last 7 day Grpah
        if data.find_all('td')[9].img is not None:
            graph = data.find_all('td')[9].find('img').attrs['src']
            WeekGraph.append(graph)
        else:
            WeekGraph.append('nan')
        dp += 1 
        clear_output(wait = True)
        print(dp)    
    browser.close()

# Converting to DataFrame

In [ ]:
pd.options.display.float_format = '{:.2f}'.format # avoid values from converting to exponent form
df = pd.DataFrame({
    "Symbols":Symbols,
    "Name":Names,
    "Categorie":Categories,
    "Price":Prices,
    "24h %":DayPercents,
    "7d %":WeekPrecets,
    "Market Cap":MarketCap,
    "Volume (24H) in  $":VDollars,
    "Volume (24H) in  Crypto":VCrypto,
    "Circulating Supply":CSupply,
    "Week Changes Graph":WeekGraph
})       
print(df.info())
df 